In [ ]:
!pip install numpy pandas scikit-learn nltk

In [ ]:
!pip install torch transformers requests

In [ ]:
!pip install huggingface_hub
!pip install --upgrade transformers


In [ ]:
pip install --upgrade --force-reinstall torch torchvision torchaudio transformers


In [ ]:
import torch
import joblib
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax

# Path to save the model
MODEL_PATH = "fact_checking_model.pkl"

def load_model():
    """
    Load the pre-trained model and tokenizer from Hugging Face.
    """
    model_name = "facebook/bart-large-mnli"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    return tokenizer, model

def save_model(model, tokenizer, model_path=MODEL_PATH):
    """
    Save the model and tokenizer as a .pkl file using joblib.
    """
    # Convert the model state_dict to a dictionary before saving
    model_data = {
        "model_state_dict": model.state_dict(),
        "tokenizer": tokenizer
    }

    joblib.dump(model_data, model_path)
    print(f"✅ Model saved successfully as {model_path}")

def load_saved_model(model_path=MODEL_PATH):
    """
    Load the saved model and tokenizer from the .pkl file.
    """
    model_name = "facebook/bart-large-mnli"

    # Load the dictionary from the pkl file
    model_data = joblib.load(model_path)

    # Reload tokenizer and model
    tokenizer = model_data["tokenizer"]
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    model.load_state_dict(model_data["model_state_dict"])
    model.eval()  # Set model to evaluation mode

    print("✅ Model loaded successfully!")
    return tokenizer, model

def predict_factuality(text, tokenizer, model):
    """
    Predict the factuality of a given text.
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = softmax(outputs.logits, dim=-1)
    labels = ["Contradiction", "Neutral", "Entailment"]
    prediction = labels[torch.argmax(probabilities).item()]
    confidence = torch.max(probabilities).item()

    return prediction, confidence

if __name__ == "__main__":
    # Load the model and save it as a .pkl file
    tokenizer, model = load_model()
    save_model(model, tokenizer)

    # Load the saved model for verification
    tokenizer, model = load_saved_model()

    # Example prediction
    test_text = "Trump is the Prime Minister of India."
    prediction, confidence = predict_factuality(test_text, tokenizer, model)
    print(f"Prediction: {prediction}, Confidence: {confidence:.2f}")


✅ Model saved successfully as fact_checking_model.pkl
✅ Model loaded successfully!
Prediction: Contradiction, Confidence: 1.00


In [ ]:
import torch
import joblib
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_PATH = "fact_checking_model6.pkl"

def save_model():
    """
    Load pre-trained model and tokenizer, then save them properly.
    """
    model_name = "facebook/bart-large-mnli"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Save tokenizer and model separately
    model_data = {
        "tokenizer_name": model_name,  # Save only model name for tokenizer
        "model_state_dict": model.state_dict()  # Save model weights
    }

    joblib.dump(model_data, MODEL_PATH)
    print(f"✅ Model saved successfully as {MODEL_PATH}")

if __name__ == "__main__":
    save_model()


✅ Model saved successfully as fact_checking_model6.pkl
